# This notebook will get rid of class 1 photos and texts and then save all the data into pickle files for the training setp

In [5]:
import os
import re
import time
import nltk
import re
import string
import tensorlayer as tl
from utils import *

In [10]:
dataset = '102flowers' #
need_256 = True # set to True for stackGAN

"""
images.shape = [8000, 64, 64, 3]
captions_ids = [80000, any]
"""
cwd = os.getcwd()
img_dir = os.path.join(cwd, '102flowers')
caption_dir = os.path.join(cwd, 'text_c10')
VOC_FIR = cwd + '/vocab.txt'

In [11]:
## load captions
caption_sub_dir = load_folder_list( caption_dir )
captions_dict = {}
processed_capts = []

In [12]:
caption_sub_dir

['C:\\jeff\\senior\\research\\text-to-image\\text_c10\\class_00001',
 'C:\\jeff\\senior\\research\\text-to-image\\text_c10\\class_00002',
 'C:\\jeff\\senior\\research\\text-to-image\\text_c10\\class_00003',
 'C:\\jeff\\senior\\research\\text-to-image\\text_c10\\class_00004',
 'C:\\jeff\\senior\\research\\text-to-image\\text_c10\\class_00005',
 'C:\\jeff\\senior\\research\\text-to-image\\text_c10\\class_00006',
 'C:\\jeff\\senior\\research\\text-to-image\\text_c10\\class_00007',
 'C:\\jeff\\senior\\research\\text-to-image\\text_c10\\class_00008',
 'C:\\jeff\\senior\\research\\text-to-image\\text_c10\\class_00009',
 'C:\\jeff\\senior\\research\\text-to-image\\text_c10\\class_00010',
 'C:\\jeff\\senior\\research\\text-to-image\\text_c10\\class_00011',
 'C:\\jeff\\senior\\research\\text-to-image\\text_c10\\class_00012',
 'C:\\jeff\\senior\\research\\text-to-image\\text_c10\\class_00013',
 'C:\\jeff\\senior\\research\\text-to-image\\text_c10\\class_00014',
 'C:\\jeff\\senior\\research\\text

In [13]:
test = "C:\\jeff\\senior\\research\\text-to-image\\text_c10\\class_00001"

In [14]:
test[-5:len(test)]

'00001'

In [15]:
for sub_dir in caption_sub_dir: # get caption file list
    # ignore if class 1
    if sub_dir[-5:len(test)] == "00001":
        print("passed")
        continue
    with tl.ops.suppress_stdout():
        files = tl.files.load_file_list(path=sub_dir, regx='^image_[0-9]+\.txt')
        for i, f in enumerate(files):
            file_dir = os.path.join(sub_dir, f)
            key = int(re.findall('\d+', f)[0])
            t = open(file_dir,'r')
            lines = []
            for line in t:
                line = preprocess_caption(line)
                lines.append(line)
                processed_capts.append(tl.nlp.process_sentence(line, start_word="<S>", end_word="</S>"))
            assert len(lines) == 10, "Every flower image have 10 captions"
            captions_dict[key] = lines
print(" * %d x %d captions found " % (len(captions_dict), len(lines)))

passed
 * 8149 x 10 captions found 


# 8189 including class 1, 8149 without

In [16]:
print(" * %d x %d captions found " % (len(captions_dict), len(lines)))

 * 8149 x 10 captions found 


In [17]:
import cv2
import numpy as np
import h5py
from PIL import Image
os.path.join(cwd)

'C:\\jeff\\senior\\research\\text-to-image'

In [18]:
f = h5py.File('C:\\jeff\\senior\\research\\text-to-image\\text_c10\\class_00001\\image_06734.h5', 'r')

In [19]:
list(f.keys())

['txt1',
 'txt10',
 'txt2',
 'txt3',
 'txt4',
 'txt5',
 'txt6',
 'txt7',
 'txt8',
 'txt9']

In [20]:
len(list(f["txt10"]))

56

In [21]:
# dset = f['txt1'][:]
# img = Image.fromarray(dset.astype('uint8'), 'RGB')
# img.save("class1/test.jpg", "JPEG")
# img.show()

In [22]:
## build vocab
if not os.path.isfile('vocab.txt'):
    _ = tl.nlp.create_vocab(processed_capts, word_counts_output_file=VOC_FIR, min_word_count=1)
else:
    print("WARNING: vocab.txt already exists")
vocab = tl.nlp.Vocabulary(VOC_FIR, start_word="<S>", end_word="</S>", unk_word="<UNK>")

INFO:tensorflow:Initializing vocabulary from file: C:\jeff\senior\research\text-to-image/vocab.txt
  [TL] Vocabulary from C:\jeff\senior\research\text-to-image/vocab.txt : <S> </S> <UNK>
    vocabulary with 5430 words (includes start_word, end_word, unk_word)
      start_id: 1
      end_id: 2
      unk_id: 5429
      pad_id: 0


In [23]:
## store all captions ids in list
captions_ids = []
try: # python3
    tmp = captions_dict.items()
except: # python3
    tmp = captions_dict.iteritems()
for key, value in tmp:
    for v in value:
        captions_ids.append( [vocab.word_to_id(word) for word in nltk.tokenize.word_tokenize(v)] + [vocab.end_id])  # add END_ID
        # print(v)              # prominent purple stigma,petals are white inc olor
        # print(captions_ids)   # [[152, 19, 33, 15, 3, 8, 14, 719, 723]]
        # exit()
captions_ids = np.asarray(captions_ids)
print(" * tokenized %d captions" % len(captions_ids))

 * tokenized 81490 captions


In [24]:
## check
img_capt = captions_dict[1][1]
print("img_capt: %s" % img_capt)
print("nltk.tokenize.word_tokenize(img_capt): %s" % nltk.tokenize.word_tokenize(img_capt))
img_capt_ids = [vocab.word_to_id(word) for word in nltk.tokenize.word_tokenize(img_capt)]#img_capt.split(' ')]
print("img_capt_ids: %s" % img_capt_ids)
print("id_to_word: %s" % [vocab.id_to_word(id) for id in img_capt_ids])

img_capt: this flower has bright purple  spiky petals  and greenish sepals below them 
nltk.tokenize.word_tokenize(img_capt): ['this', 'flower', 'has', 'bright', 'purple', 'spiky', 'petals', 'and', 'greenish', 'sepals', 'below', 'them']
img_capt_ids: [6, 3, 7, 31, 18, 165, 4, 5, 318, 83, 374, 120]
id_to_word: ['this', 'flower', 'has', 'bright', 'purple', 'spiky', 'petals', 'and', 'greenish', 'sepals', 'below', 'them']


In [25]:
len(captions_dict)

8149

In [26]:
## load images
with tl.ops.suppress_stdout():  # get image files list
    imgs_title_list = sorted(tl.files.load_file_list(path=img_dir, regx='^image_[0-9]+\.jpg'))
print(" * %d images found, start loading and resizing ..." % len(imgs_title_list))

 * 8189 images found, start loading and resizing ...


In [27]:
name = "image_06734.jpg"

if int(name[6:11]) >= 6734 and int(name[6:11]) <= 6773:
    print(True)

True


In [21]:
s = time.time()

# time.sleep(10)
# def get_resize_image(name):   # fail
#         img = scipy.misc.imread( os.path.join(img_dir, name) )
#         img = tl.prepro.imresize(img, size=[64, 64])    # (64, 64, 3)
#         img = img.astype(np.float32)
#         return img
# images = tl.prepro.threading_data(imgs_title_list, fn=get_resize_image)
images = []
images_256 = []
for name in imgs_title_list:
    if int(name[6:11]) >= 6734 and int(name[6:11]) <= 6773:
        continue
    img_raw = scipy.misc.imread( os.path.join(img_dir, name) )
    img = tl.prepro.imresize(img_raw, size=[64, 64])    # (64, 64, 3)
    img = img.astype(np.float32)
    images.append(img)
    if need_256:
        img = tl.prepro.imresize(img_raw, size=[256, 256]) # (256, 256, 3)
        img = img.astype(np.float32)

        images_256.append(img)
# images = np.array(images)
# images_256 = np.array(images_256)
print(" * loading and resizing took %ss" % (time.time()-s))

C:\Users\cseuser\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  from ipykernel import kernelapp as app


 * loading and resizing took 117.95992136001587s


In [79]:
n_images = len(captions_dict)
n_captions = len(captions_ids)
n_captions_per_image = len(lines) # 10

print("n_captions: %d n_images: %d n_captions_per_image: %d" % (n_captions, n_images, n_captions_per_image))

n_captions: 81490 n_images: 8149 n_captions_per_image: 10


In [80]:
captions_ids_train, captions_ids_test = captions_ids[: 8000*n_captions_per_image], captions_ids[8000*n_captions_per_image :]
images_train, images_test = images[:8000], images[8000:]
if need_256:
    images_train_256, images_test_256 = images_256[:8000], images_256[8000:]
n_images_train = len(images_train)
n_images_test = len(images_test)
n_captions_train = len(captions_ids_train)
n_captions_test = len(captions_ids_test)
print("n_images_train:%d n_captions_train:%d" % (n_images_train, n_captions_train))
print("n_images_test:%d  n_captions_test:%d" % (n_images_test, n_captions_test))

## check test image
# idexs = get_random_int(min=0, max=n_captions_test-1, number=64)
# temp_test_capt = captions_ids_test[idexs]
# for idx, ids in enumerate(temp_test_capt):
#     print("%d %s" % (idx, [vocab.id_to_word(id) for id in ids]))
# temp_test_img = images_train[np.floor(np.asarray(idexs).astype('float')/n_captions_per_image).astype('int')]
# save_images(temp_test_img, [8, 8], 'temp_test_img.png')
# exit()

# ## check the first example
# tl.visualize.frame(I=images[0], second=5, saveable=True, name='temp', cmap=None)
# for cap in captions_dict[1]:
#     print(cap)
# print(captions_ids[0:10])
# for ids in captions_ids[0:10]:
#     print([vocab.id_to_word(id) for id in ids])
# print_dict(captions_dict)

# ## generate a random batch
# batch_size = 64
# idexs = get_random_int(0, n_captions_test, batch_size)
# # idexs = [i for i in range(0,100)]
# print(idexs)
# b_seqs = captions_ids_test[idexs]
# b_images = images_test[np.floor(np.asarray(idexs).astype('float')/n_captions_per_image).astype('int')]
# print("before padding %s" % b_seqs)
# b_seqs = tl.prepro.pad_sequences(b_seqs, padding='post')
# print("after padding %s" % b_seqs)
# # print(input_images.shape)   # (64, 64, 64, 3)
# for ids in b_seqs:
#     print([vocab.id_to_word(id) for id in ids])
# print(np.max(b_images), np.min(b_images), b_images.shape)
# from utils import *
# save_images(b_images, [8, 8], 'temp2.png')
# # tl.visualize.images2d(b_images, second=5, saveable=True, name='temp2')
# exit()

# import pickle
from sklearn.externals import joblib
def save_all(targets, file):
    with open(file, 'wb') as f:
        joblib.dump(targets, f)
        # pickle.dump(targets, f)

save_all(vocab, '_vocab.sav')
save_all((images_train_256, images_train), '_image_train.sav')
save_all((images_test_256, images_test), '_image_test.sav')
save_all((n_captions_train, n_captions_test, n_captions_per_image, n_images_train, n_images_test), '_n.sav')
save_all((captions_ids_train, captions_ids_test), '_caption.sav')

n_images_train:8000 n_captions_train:80000
n_images_test:149  n_captions_test:1490
